## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 

# Add a variable for the files to load
city_data_to_load = os.path.join("..\Weather_Database","WeatherPy_Database.csv")

# Read the City Data
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Ancud,CL,-41.8697,-73.8203,42.33,78,4,7.20,clear sky
1,1,Provideniya,RU,64.3833,-173.3000,47.21,94,100,4.76,light rain
2,2,Cheney,US,47.4874,-117.5758,104.76,16,75,9.22,broken clouds
3,3,Mitsamiouli,KM,-11.3847,43.2844,76.32,68,0,6.26,clear sky
4,4,Korla,CN,41.7597,86.1469,84.04,29,93,12.28,overcast clouds
5,5,Krasnoselkup,RU,65.7000,82.4667,53.47,68,63,6.29,broken clouds
6,6,Dikson,RU,73.5069,80.5464,43.29,98,98,7.56,overcast clouds
7,7,Aksarka,RU,66.5606,67.7975,47.37,91,100,9.66,overcast clouds
8,8,Tiksi,RU,71.6872,128.8694,47.62,75,51,7.78,broken clouds
9,9,Ribeira Grande,PT,38.5167,-28.7000,71.38,85,13,6.82,few clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 150


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
2,2,Cheney,US,47.4874,-117.5758,104.76,16,75,9.22,broken clouds
3,3,Mitsamiouli,KM,-11.3847,43.2844,76.32,68,0,6.26,clear sky
4,4,Korla,CN,41.7597,86.1469,84.04,29,93,12.28,overcast clouds
5,5,Krasnoselkup,RU,65.7000,82.4667,53.47,68,63,6.29,broken clouds
9,9,Ribeira Grande,PT,38.5167,-28.7000,71.38,85,13,6.82,few clouds
10,10,Miri,MY,4.4148,114.0089,74.44,89,74,5.41,broken clouds
11,11,Flagstaff,US,35.1981,-111.6513,76.32,86,40,3.44,scattered clouds
12,12,Broome,US,42.2506,-75.8330,66.69,65,1,4.00,clear sky
13,13,Victoria,HK,22.2855,114.1577,84.38,91,85,8.99,light rain
14,14,Bathsheba,BB,13.2167,-59.5167,83.01,78,40,14.97,scattered clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                592
City                   592
Country                588
Lat                    592
Lng                    592
Max Temp               592
Humidity               592
Cloudiness             592
Wind Speed             592
Current description    592
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.count()

City_ID                588
City                   588
Country                588
Lat                    588
Lng                    588
Max Temp               588
Humidity               588
Cloudiness             588
Wind Speed             588
Current description    588
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
2,Cheney,US,104.76,broken clouds,47.4874,-117.5758,
3,Mitsamiouli,KM,76.32,clear sky,-11.3847,43.2844,
4,Korla,CN,84.04,overcast clouds,41.7597,86.1469,
5,Krasnoselkup,RU,53.47,broken clouds,65.7000,82.4667,
9,Ribeira Grande,PT,71.38,few clouds,38.5167,-28.7000,
10,Miri,MY,74.44,broken clouds,4.4148,114.0089,
11,Flagstaff,US,76.32,scattered clouds,35.1981,-111.6513,
12,Broome,US,66.69,clear sky,42.2506,-75.8330,
13,Victoria,HK,84.38,light rain,22.2855,114.1577,
14,Bathsheba,BB,83.01,scattered clouds,13.2167,-59.5167,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    hotels
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [17]:
# 7. Drop the rows where there is no Hotel Name.
print(len(hotel_df))
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"]==""].index)
print(len(clean_hotel_df))

588
537


In [18]:
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [23]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
2,Cheney,US,104.76,broken clouds,47.4874,-117.5758,"Holiday Inn Express & Suites Cheney, an IHG Hotel"
3,Mitsamiouli,KM,76.32,clear sky,-11.3847,43.2844,Foyer ADM
4,Korla,CN,84.04,overcast clouds,41.7597,86.1469,Gulisitan Hotel
5,Krasnoselkup,RU,53.47,broken clouds,65.7000,82.4667,Krasnosel'kupnefteprodukt
9,Ribeira Grande,PT,71.38,few clouds,38.5167,-28.7000,Quinta da Meia Eira
10,Miri,MY,74.44,broken clouds,4.4148,114.0089,"Mega Hotel, Miri"
11,Flagstaff,US,76.32,scattered clouds,35.1981,-111.6513,Drury Inn & Suites Flagstaff
12,Broome,US,66.69,clear sky,42.2506,-75.8330,Chenango Valley State Park
13,Victoria,HK,84.38,light rain,22.2855,114.1577,Mini Hotel Central
14,Bathsheba,BB,83.01,scattered clouds,13.2167,-59.5167,Atlantis Hotel


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current weather</dt><dd>{Current description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))